<a href="https://colab.research.google.com/github/Lilwm/Optimizing-DataFrames-and-Processing-in-Chunks/blob/main/Optimizing_DataFrames_and_Processing_in_Chunks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Notebook: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [8]:
# Importing pandas
import pandas as pd
pd.options.display.max_columns = 99

# Your code goes here
loans_df = pd.read_csv('https://bit.ly/3H2XVgC')
loans_df.head()

#Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows 
loans_df2 = pd.read_csv('https://bit.ly/3H2XVgC', nrows=1000)
print("usage(MB) for 1000 rows: ",loans_df2.memory_usage(deep=True).sum()/(1024**2))  

usage(MB) for 1000 rows:  1.5273666381835938


In [9]:
# Increase the number of rows 3 times
loans_df = pd.read_csv('https://bit.ly/3H2XVgC', nrows=3000)
print(loans_df.memory_usage(deep=True).sum() / (1024**2), 'MB')

4.580394744873047 MB


*loading the first 3000 rows seems to keep the memory usage under 5 MB*

In [10]:
loan_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000)

for chunk in loan_iter:
  print(chunk.memory_usage(deep=True).sum() / (1024**2), 'MB')


4.580394744873047 MB
4.576141357421875 MB
4.577898979187012 MB
4.579251289367676 MB
4.575444221496582 MB
4.577326774597168 MB
4.575918197631836 MB
4.578287124633789 MB
4.576413154602051 MB
4.57646369934082 MB
4.589176177978516 MB
4.588043212890625 MB
4.594850540161133 MB
4.828314781188965 MB
0.868586540222168 MB


*All the chunks of our dataset consume less than 5 MB as required*

## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

In [24]:
loan_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000)

numeric = []
string = []
for chunk in loan_iter:
    # Calculate amount of columns for each type per chunk
    numeric.append(chunk.select_dtypes(include='number').shape[1])
    string.append(chunk.select_dtypes(include='object').shape[1])
    
print(f'Numeric columns per chunk: \n{numeric}')

print(f'\nString columns per chunk:\n {string}')

Numeric columns per chunk: 
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]

String columns per chunk:
 [21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


In [12]:
obj_cols = []
loan_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000)

for chunk in loan_iter:
    chunk_obj_cols = chunk.select_dtypes(include=['object']).columns.tolist()
    if len(obj_cols) > 0:
        similar = obj_cols == chunk_obj_cols
        if not similar:
            print(f'overall string cols: \n {obj_cols}')
            print(f'chunk string cols: \n{chunk_obj_cols}')    
    else:
        obj_cols = chunk_obj_cols

overall string cols:
 ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

chunk string cols:
 ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

overall string cols:
 ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'applicati

### Observations 
*   There are 31 numeric columns and 21 string columns in most chunks except the last 2
*  It seems like the *id column* is being cast to int64 in the last 2 chunks but not in the earlier chunks. Since the id column won't be useful for analysis we can ignore this column.



In [13]:
#check for string values less than 50%
loan_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000)

unique_percent =[]
for chunk in loan_iter:
  chunk['id'] = pd.to_numeric(chunk['id'], errors='coerce') #non-numeric string converted to null values
  chunk = chunk.dropna(axis=0, subset=['id']) #drop row where id is missing
  #check unique values per col
  string_cols_unique = chunk.select_dtypes(include=['object']).nunique() 
  string_cols_count = chunk.select_dtypes(include=['object']).count() # get total no of items in col
  unique_percent.append((100*string_cols_unique/string_cols_count))  # get unique%

  total_unique = pd.concat(unique_percent)
  total_unique = total_unique.groupby(total_unique.index).mean()
  string_cols_categorical = list((total_unique[total_unique <50]).index)

print(f'\npercent of unique values:  \n{total_unique}')
print(string_cols_categorical)


percent of unique values:  
addr_state              1.880203
application_type        0.043550
earliest_cr_line       14.686817
emp_length              0.488414
emp_title              92.775698
grade                   0.304848
home_ownership          0.156422
initial_list_status     0.043550
int_rate                3.376513
issue_d                 0.343083
last_credit_pull_d      3.309096
last_pymnt_d            2.488140
loan_status             0.149323
purpose                 0.579480
pymnt_plan              0.045772
revol_util             34.412441
sub_grade               1.517572
term                    0.070217
title                  64.571386
verification_status     0.105773
zip_code               20.765791
dtype: float64
['addr_state', 'application_type', 'earliest_cr_line', 'emp_length', 'grade', 'home_ownership', 'initial_list_status', 'int_rate', 'issue_d', 'last_credit_pull_d', 'last_pymnt_d', 'loan_status', 'purpose', 'pymnt_plan', 'revol_util', 'sub_grade', 'term', 'verific

In [14]:
#Float columns with no missing values
loan_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000)

float_null_count =[]
for chunk in loan_iter:
  chunk['id'] = pd.to_numeric(chunk['id'], errors='coerce') #non-numeric string converted to null values
  chunk = chunk.dropna(axis=0, subset=['id']) #drop row where id is missing
  float_cols_null = chunk.select_dtypes(exclude=['object']).isnull().sum()
  float_null_count.append(float_cols_null)
  
  #combine all numeric columns missing values
  total_float_null = pd.concat(float_null_count)
  total_float_null = total_float_null.groupby(total_float_null.index).sum()
  no_missing = set((total_float_null[total_float_null == 0 ]).index)
  missing = set((total_float_null[total_float_null > 0 ]).index)

print(f'float columns with missing values:  \n{missing}')
print(f'\nfloat columns with no missing values:  \n{no_missing}')


float columns with missing values:  
{'pub_rec', 'open_acc', 'chargeoff_within_12_mths', 'delinq_2yrs', 'delinq_amnt', 'collections_12_mths_ex_med', 'inq_last_6mths', 'total_acc', 'pub_rec_bankruptcies', 'tax_liens', 'annual_inc', 'acc_now_delinq'}

float columns with no missing values:  
{'out_prncp_inv', 'recoveries', 'out_prncp', 'total_rec_prncp', 'last_pymnt_amnt', 'funded_amnt_inv', 'member_id', 'loan_amnt', 'dti', 'total_pymnt', 'policy_code', 'total_pymnt_inv', 'collection_recovery_fee', 'installment', 'funded_amnt', 'total_rec_late_fee', 'total_rec_int', 'id', 'revol_bal'}


In [15]:
# total memory usage 
loans_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000)

initial_memory = []
for chunk in loans_iter:
    initial_memory.append(chunk.memory_usage(deep=True).sum() / (1024 ** 2))
    
print('Total memory usage: {:.4f} MB'.format(sum(initial_memory)))

Total memory usage: 65.2425 MB


*Initial memory usage is 65.24MB. This will be our reference when lowering the amount of total memory usage.*

## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [23]:
# we can convert the above  object columns into the following dtypes
       # To Category: [sub_grade, 'home_ownership', 'verification_status','purpose']
        # To numerical: ['term', 'revol_util', 'int_rate']
        # To datetime:['issue_d', 'earliest_cr_line','last_pymnt_d','last_credit_pull_d']
convert_col_dtypes = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}
#pass the dtypes when reading the df
loan_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000, dtype=convert_col_dtypes, 
                         parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])
total_memory = []
for chunk in loan_iter:
    # clean columns to convert to numeric
    term_cleaned = chunk['term'].str.lstrip(" ").str.rstrip(" months")  #remove months 
    int_rate_cleaned = chunk['int_rate'].str.rstrip("%")  #remove % 
    revol_cleaned = chunk['revol_util'].str.rstrip("%") #remove % 
    #convert  to numeric data types
    chunk['term'] = pd.to_numeric(term_cleaned)
    chunk['revol_util'] = pd.to_numeric(revol_cleaned)
    chunk['int_rate'] = pd.to_numeric(int_rate_cleaned)
    total_memory.append(chunk.memory_usage(deep=True).sum() / (1024 ** 2))
    
# print(chunk.dtypes)
print(f'Total memory usage with string optimization: {sum(total_memory)}')


Total memory usage with string optimization: 38.81328868865967


### Observation
*Memory usage improved from **65.25MB to 38.8MB** after changing data to categorical*

## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




**float to more efficient subtype conversion**

in this step, we deal with the columns that have missing values

In [17]:
#check for NaN values
loans_df.isna().sum()

id                              0
member_id                       0
loan_amnt                       0
funded_amnt                     0
funded_amnt_inv                 0
term                            0
int_rate                        0
installment                     0
grade                           0
sub_grade                       0
emp_title                     171
emp_length                     83
home_ownership                  0
annual_inc                      0
verification_status             0
issue_d                         0
loan_status                     0
pymnt_plan                      0
purpose                         0
title                           0
zip_code                        0
addr_state                      0
dti                             0
delinq_2yrs                     0
earliest_cr_line                0
inq_last_6mths                  0
open_acc                        0
pub_rec                         0
revol_bal                       0
revol_util    

In [19]:
# Your code goes here

loan_iter = pd.read_csv('https://bit.ly/3H2XVgC', chunksize=3000, dtype=convert_col_dtypes, 
                         parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"], )
import numpy as np
#function to convert to int for cols with no missing values
def change_to_int(df, col_name):
    # Get the min and max values
    max_col = df[col_name].max()
    min_col = df[col_name].min()
    # Find the datatype
    for dtype_name in ['int8', 'int16', 'int32', 'int64']:
        # Check if this datatype can hold all values
        if max_col <  np.iinfo(dtype_name).max and min_col > np.iinfo(dtype_name).min:
            df[col_name] = df[col_name].astype(dtype_name)
            break

total_memory = 0
for chunk in loan_iter:
  #convert the id column object to float and remove null value
  chunk['id'] = pd.to_numeric(chunk['id'], errors='coerce') #non-numeric string converted to null values
  chunk = chunk.dropna(axis=0, subset=['id']) #drop row where id is missing
  term_cleaned = chunk['term'].str.lstrip(" ").str.rstrip(" months")  #remove months to convert to int
  int_rate_cleaned = chunk['int_rate'].str.rstrip("%")
  revol_cleaned = chunk['revol_util'].str.rstrip("%") #remove % 
  chunk['term'] = pd.to_numeric(term_cleaned)
  chunk['revol_util'] = pd.to_numeric(revol_cleaned)
  chunk['int_rate'] = pd.to_numeric(int_rate_cleaned)

  float_cols = chunk.select_dtypes(include=['float'])
  float_cols = float_cols.dropna()
  for col in float_cols.columns:
    if col in missing:
      chunk[col] = pd.to_numeric(chunk[col], downcast='float')
    elif col in no_missing:
      change_to_int(chunk, col)
  #calculate memory usage
  total_memory += chunk.memory_usage(deep=True).sum() / (1024 ** 2)

print(f'\nTotal memory usage: {total_memory}')
print(f'\nPercentage memory savings: {(100*(sum(initial_memory) - (total_memory))/sum(initial_memory)}')
print(f'\n {chunk.dtypes}')


Total memory usage: 32.99 MB

Percentage memory savings: 49.44 %

 id                                     int32
member_id                              int32
loan_amnt                              int16
funded_amnt                            int16
funded_amnt_inv                        int16
term                                   int64
int_rate                             float64
installment                            int16
grade                                 object
sub_grade                           category
emp_title                             object
emp_length                            object
home_ownership                      category
annual_inc                           float32
verification_status                 category
issue_d                       datetime64[ns]
loan_status                           object
pymnt_plan                            object
purpose                             category
title                                 object
zip_code                        

*we managed to achieve 49.44% savings on memory. By further optimizing the integer data types, memory usage dropped to ~33MB from 65MB*

## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [20]:
# Your code goes here
# Determine the optimal chunk size based on the memory constraints you provide.

def optimal_chunk_size(csv_file, desired_mem, row_steps, start_chunk_size):
  no_rows = start_chunk_size
  chunk_memory =0
  while(chunk_memory < desired_mem):
    no_rows += row_steps
    chunk = pd.read_csv(csv_file, nrows=no_rows)
    chunk_memory = chunk.memory_usage(deep=True).sum()/(1024**2)
  return(no_rows - row_steps) # reduce by 1

optimal_chunk_size('https://bit.ly/3H2XVgC', 5, 50, 2500)

3250

In [26]:
# Determine which string columns can be converted to numeric ones by removing the % character
def string_to_numeric(df, character):
  columns_with_char = []
  for key, value in df.iteritems():
    if  value.str.contains(character).any(): #check if column has % character
      try:
        value = value.str.replace('%', '')  #remove it and try to convert to numeric
        value = pd.to_numeric(value, errors='raise') 
        columns_with_char.append(key)
      except(ValueError):
        #dont add columns to the list
        continue
  print(columns_with_char)

loans = pd.read_csv('https://bit.ly/3H2XVgC')
string_cols = loans.select_dtypes(include='object')
string_to_numeric(string_cols, '%')

['int_rate', 'revol_util']


In [27]:
#Determine which numeric columns can be converted to more space efficient representations

# int64(no missing value) or float64(allow missing value)
def numeric_mem_optimise(df): 
  #select float type columns and check the number of missing value per columns
  numeric_cols = df.select_dtypes(exclude=['object'])
  cols_missing_value = numeric_cols.isnull().sum()
  #float columns with no null values can be converted to int type
  int_cols = list(cols_missing_value[cols_missing_value == 0].index)
  float_cols = list(cols_missing_value[cols_missing_value >0].index)

  return int_cols, float_cols

#
loans = pd.read_csv('https://bit.ly/3H2XVgC')
loans['id']=pd.to_numeric(loans['id'], errors='coerce')
loans.dropna(axis=0, subset=['id'], inplace=True)

x,y = numeric_mem_optimise(loans)
print(f'int columns: {x}')
print(f'float columns: {y}')

int columns: ['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'installment', 'dti', 'revol_bal', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'policy_code']
float columns: ['annual_inc', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'total_acc', 'collections_12_mths_ex_med', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies', 'tax_liens']
